## This notebook walks through RAG approach to demonstrate the integration of watsonx.ai and watsonx.data platform through MILVUS. 
The following steps:

1- Loading data from documents into watsonx.data

2- Integrating milvus vector database to watsonx.data and loading vector embeddings to milvus

3- Querying milvus and invoking LLM to get results

#### Install required libraries and load documents

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Restart kernel after installing the packages

In [2]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
pdf_files=['/Users/pratiksinha/Desktop/workshop/RAG with watsonx.data using milvus/pdf/IBM Watsonx.Data [TechTalks].pdf',
          '/Users/pratiksinha/Desktop/workshop/RAG with watsonx.data using milvus/pdf/DB2 and DB2 Warehouse on Cloud [TechTalks].pdf',
          '/Users/pratiksinha/Desktop/workshop/RAG with watsonx.data using milvus/pdf/IBM watsonx.data Milvus Vector Database Competitive - Mar 2024 Final.pdf',
          '/Users/pratiksinha/Desktop/workshop/RAG with watsonx.data using milvus/pdf/watsonx_Value, Differentiators and Capabilities.pdf']

In [4]:
all_data=''
if pdf_files:
    for pdf in pdf_files:
        pdf_reader = PyPDF2.PdfReader(pdf)
        # Extract text from each page
        for page in pdf_reader.pages:
            text = page.extract_text()
            all_data += text
all_data = all_data.replace("'", "''").replace("%", "%%").replace("\n"," ")

In [5]:
len(all_data)

93671

## Load document into watsonx.data 

#### Connect to watsonx.data 

In [6]:
import ssl
import urllib3
import os
from sqlalchemy import create_engine
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) # disable https warning

LH_HOST_NAME='useast.services.cloud.techzone.ibm.com'
LH_PORT=33027
LH_USER='ibmlhadmin'
LH_PW='password'
LH_CATALOG='tpch'
LH_SCHEMA='tiny'

quick_engine = create_engine(
   f"presto://{LH_USER}:{LH_PW}@{LH_HOST_NAME}:{LH_PORT}/{LH_CATALOG}/{LH_SCHEMA}",
   connect_args={
    'protocol': 'https', 
    'requests_kwargs': {'verify': ssl.CERT_NONE }
    }
)

In [7]:
## optional connection check (this may take several seconds)
with quick_engine.connect() as conn:
    df = pd.read_sql(
        sql='select * from tpch.tiny.customer limit 10',
        con=conn.connection
    )
df

,custkey,name,address,nationkey,phone,acctbal,mktsegment,comment
0,376,Customer#000000376,4NwsvFQU T4mSgzvU1Rx2ZtHOGyaNyhe,16,26-437-952-8986,4231.45,AUTOMOBILE,gs cajole quickly. bold asymptotes wake regula...
1,377,Customer#000000377,"PA4levhyD,Rvr0JHQ4QNOqJ9gW YXE",23,33-260-610-4079,1043.72,MACHINERY,. slyly regular ideas cajole blithely. slyly i...
2,378,Customer#000000378,"133stqM9 LT,a2BSlbm49 nXreFggaZgW6P6J",22,32-147-793-4825,5718.05,BUILDING,ackages haggle fluffily ironic packages.
3,379,Customer#000000379,"t3QzCf,q1NbshmjOIUY",7,17-228-550-9246,5348.11,AUTOMOBILE,l deposits cajole blithely blithely final depo...
4,380,Customer#000000380,n2w3Jd1bipwICbOVgrELzcNRexmWSklo,21,31-538-493-4229,2755.46,BUILDING,riously special accounts. slyly final accounts...
5,381,Customer#000000381,w3zVseYDbjBbzLld,5,15-860-208-7093,9931.71,BUILDING,"t regular, bold accounts. carefully quick pack..."
6,382,Customer#000000382,"UdgAMamK5JnSykA,ZPfR5W5zRFatDUye",8,18-942-650-6657,6269.42,AUTOMOBILE,. blithely express notornis according to the b...
7,383,Customer#000000383,iBIHYgXvVDpu6qq7FlqXVcAIDAzv4qs,2,12-868-920-9034,-849.44,MACHINERY,slyly express ideas haggle blithely unusual du...
8,384,Customer#000000384,kDDMb3ML nUSu2Sn7CUHyZVedAFUx9,9,19-271-453-8361,-614.30,HOUSEHOLD,"olites. express, unusual dolphins cajole caref..."
9,385,Customer#000000385,zJvPI24TSPpiFzYfu3RvTKQ9,3,13-741-675-6890,2457.09,AUTOMOBILE,rs. blithely ironic deposits nag furiously acr...


### Create Schema in watsonx.data Hive Bucket to store data

In [15]:
with quick_engine.connect() as conn:
    create_schema_result = pd.read_sql(sql="""
    CREATE SCHEMA hive_data.pratikmilvus WITH ( location = 's3a://hive-bucket/pratikmilvus')
    """,         
    con=conn.connection
)

In [16]:
# Create table 
with quick_engine.connect() as conn:
    create_table_result = pd.read_sql(sql="""
    CREATE TABLE hive_data.pratikmilvus.watsonwise
      (
      "id" varchar,
      "text" varchar,
      "title" varchar  )
     WITH (
     format = 'PARQUET',
     external_location = 's3a://hive-bucket/pratikmilvus'
     )     
    """,
    con=conn.connection
)

In [17]:
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP
        )
texts = text_splitter.split_text(all_data)

In [18]:
# Insert data to watsonx.data
for index, i in enumerate(texts):
            
            chunk = texts[index].replace("'", "''").replace("%", "%%").replace("\n"," ")
            insert_stmt = f"insert into hive_data.pratikmilvus.watsonwise values ('{index+1}', '{chunk}', '{pdf}')"
            
            with quick_engine.connect() as connection:
                connection.execute(insert_stmt)
            print(f"{pdf} chunk {index+1} INSERTED")
print('Data inserted to watsonx.data')

/Users/pratiksinha/Desktop/milvus/watsonx_Value, Differentiators and Capabilities.pdf chunk 1 INSERTED
/Users/pratiksinha/Desktop/milvus/watsonx_Value, Differentiators and Capabilities.pdf chunk 2 INSERTED
/Users/pratiksinha/Desktop/milvus/watsonx_Value, Differentiators and Capabilities.pdf chunk 3 INSERTED
/Users/pratiksinha/Desktop/milvus/watsonx_Value, Differentiators and Capabilities.pdf chunk 4 INSERTED
/Users/pratiksinha/Desktop/milvus/watsonx_Value, Differentiators and Capabilities.pdf chunk 5 INSERTED
/Users/pratiksinha/Desktop/milvus/watsonx_Value, Differentiators and Capabilities.pdf chunk 6 INSERTED
/Users/pratiksinha/Desktop/milvus/watsonx_Value, Differentiators and Capabilities.pdf chunk 7 INSERTED
/Users/pratiksinha/Desktop/milvus/watsonx_Value, Differentiators and Capabilities.pdf chunk 8 INSERTED
/Users/pratiksinha/Desktop/milvus/watsonx_Value, Differentiators and Capabilities.pdf chunk 9 INSERTED
/Users/pratiksinha/Desktop/milvus/watsonx_Value, Differentiators and Capa

In [8]:
# confirm data inserted
with quick_engine.connect() as conn:
    articles = pd.read_sql(
        sql='select * from hive_data.pratikmilvus.watsonwise limit 10',
        con=conn.connection
    )
articles

,id,text,title
0,31,data in motion and at restFIPS 140-2 certified...,"watsonx_Value, Differentiators and Capabilitie..."
1,32,the month1. Start with a base instance for ste...,"watsonx_Value, Differentiators and Capabilitie..."
2,113,provides the most vector index types to acc...,"watsonx_Value, Differentiators and Capabilitie..."
3,15,14Top Use-cases for Db2Warehouse on Cloud Reta...,"watsonx_Value, Differentiators and Capabilitie..."
4,97,search between the query vector and the vector...,"watsonx_Value, Differentiators and Capabilitie..."
5,55,editor and re-evaluate quickly and easilyRun S...,"watsonx_Value, Differentiators and Capabilitie..."
6,16,Cloud ArchitectureEnrichsensitiveon-premisesda...,"watsonx_Value, Differentiators and Capabilitie..."
7,114,"knowledge, facts, and skills pluggable for LLM...","watsonx_Value, Differentiators and Capabilitie..."
8,71,lakes Run existing reporting and enable new A...,"watsonx_Value, Differentiators and Capabilitie..."
9,99,searches•The lower layers provide most of the ...,"watsonx_Value, Differentiators and Capabilitie..."


#### Load credentials for `ibm-watson-machine-learning` 


In [9]:
import os
from dotenv import load_dotenv
from ibm_cloud_sdk_core import IAMTokenManager

load_dotenv('config.env')

# Connection variables
api_key = os.getenv("API_KEY", None)               #<your-ibm-cloud-api-key>
ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None)   #<your-ibm-cloud-url>
project_id = os.getenv("PROJECT_ID", None)         #<your-watsonx.ai-project-id>
creds = {
    "url": ibm_cloud_url,
    "apikey": api_key 
}
access_token = IAMTokenManager(
    apikey = api_key,
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

#### Create Lakehouse Connection

We will use this watsonx.data connection to load the document.

In [10]:
import ssl
import urllib3
import os
from sqlalchemy import create_engine
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) # disable https warning

LH_HOST_NAME=os.getenv("LH_HOST_NAME", None)
LH_PORT=os.getenv("LH_PORT", None) 
LH_USER=os.getenv("LH_USER", None)
LH_PW=os.getenv("LH_PW", None)
LH_CATALOG='tpch'
LH_SCHEMA='tiny'
LH_CERTFILE='cert.crt'

try: 
    quick_engine.dispose()
except:
    pass


quick_engine = create_engine(
   f"presto://{LH_USER}:{LH_PW}@{LH_HOST_NAME}:{LH_PORT}/{LH_CATALOG}/{LH_SCHEMA}",
   connect_args={
    'protocol': 'https', 
    'requests_kwargs': {'verify': f'{LH_CERTFILE}'}
    }
)

## Load Vector Embeddings to Milvus

Here we will take the data we loaded into watsonx.data from the previous step and load it into the vector database Milvus. This data was previously chunked and stored in a watsonx.data hive table, so we'll pull from here, vectorize the text chunks and load them into Milvus.

Before we can start loading the data, though, we need to create a collection in Milvus to hold the data. We'll call this collection `watsonx`. This collection holds the vector embeddings for each chunk of text, as well as the original text itself and additional context.

Let's get started!

#### Create Milvus Collection & Index

Creating a Milvus collection involves first connecting to the Milvus server, then creating a collection with a defined schema and index. 

In [11]:
from pymilvus import(
    Milvus,
    IndexType,
    Status,
    connections,
    FieldSchema,
    DataType,
    Collection,
    CollectionSchema,
)

import os 

host = os.getenv("MILVUS_HOST", None)
port = os.getenv("MILVUS_PORT", None)
password = 'password'
user = 'ibmlhadmin'
server_pem_path = 'cert.crt'

connections.connect(alias = 'default',
                   host = host,
                   port = port,
                   user = user,
                   password = password,
                   server_pem_path = server_pem_path,
                   server_name = 'watsonxdata',
                   secure = True)

In [13]:
# Create collection - define fields + schema

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True), # Primary key
    FieldSchema(name="article_text", dtype=DataType.VARCHAR, max_length=2500,),
    FieldSchema(name="article_title", dtype=DataType.VARCHAR, max_length=200,),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=384),
]

schema = CollectionSchema(fields, "watsonx collection schema")

collection = Collection("pratikmilvus", schema)

# Create index
index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":2048}
}

collection.create_index(field_name="vector", index_params=index_params)

Status(code=0, message=)

In [12]:
## Status(code=0, message=) means success! 

In [14]:
# we can run a check to see the collections in our milvus instance and we see if our collection has been created 

from pymilvus import utility
utility.list_collections()

['pratik', 'watsonx', 'pratikmilvus']

#### Insert Vectors into Milvus

Here we read data from the lakehouse table using the connection we created earlier. We pull text chunks and titles from the database, being sure to separate them out into separate lists. We then vectorize using the `sentence-transformers/all-MiniLM-L6-v2` sentence transformer model. Learn more about Hugging Face sentence transformers here: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

It is important we assemble the article text, article titles and vector embeddings into a `data` object. This object will be used to load the data into Milvus.

In [15]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from pymilvus import Collection, connections
import warnings
warnings.filterwarnings('ignore')

# Download articles from watsonx.data using the engine we created earlier 
with quick_engine.connect() as conn:
    articles_df = pd.read_sql(
        sql='select * from hive_data.pratikmilvus.watsonwise',
        con=conn.connection
    )

# extract text + titles
passages = articles_df['text'].tolist()
passage_titles = articles_df['title'].tolist()

# Create vector embeddings + data
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') # 384 dim
passage_embeddings = model.encode(passages)

basic_collection = Collection("pratikmilvus") 
data = [
    passages,
    passage_titles,
    passage_embeddings
]
out = basic_collection.insert(data)
basic_collection.flush()  # Ensures data persistence

In [16]:
## check to ensure entities have been loaded into the collection

basic_collection = Collection("pratikmilvus") 
basic_collection.num_entities 

210

## Query Milvus & Prompt LLM

After gathering the data from document and then vectorizing it and inserting into Milvus, we are now ready to perform queries against the vector database. We will use the same `sentence-transformers/all-MiniLM-L6-v2` model to generate the query vector and then use Milvus to find the most similar vectors in the database.

#### Load Milvus Collection 

In [19]:
# Load collection
basic_collection = Collection("pratikmilvus")      
basic_collection.load()

# Query function
def query_milvus(query, num_results=5):
    
    # Vectorize query
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') # 384 dim
    query_embeddings = model.encode([query])

    # Search
    search_params = {
        "metric_type": "L2", 
        "params": {"nprobe": 5}
    }
    results = basic_collection.search(
        data=query_embeddings, 
        anns_field="vector", 
        param=search_params,
        limit=num_results,
        expr=None, 
        output_fields=['article_text'],
    )
    return results

#### Prompt LLM with Query Results

In [20]:
#Prompt questions for Query results
question_text = "List some core functionalities of watsonx.data in 500 words?"
#question_text = " Explain watsonx.data in 1000 words? "

In [21]:
# Query Milvus 

num_results = 3
results = query_milvus(question_text, num_results)

relevant_chunks = []
for i in range(num_results):    
    #print(f"id: {results[0].ids[i]}")
    #print(f"distance: {results[0].distances[i]}")
    text = results[0][i].entity.get('article_text')
    relevant_chunks.append(text)
    
#print(relevant_chunks)

In [22]:
def make_prompt(context, question_text):
    return (f"{context}\n\nPlease answer a question using this text. "
          + f"If the question is unanswerable, say \"unanswerable\"."
          + f"\n\nQuestion: {question_text}")


# Build prompt w/ Milvus results
# Embed retrieved passages(context) and user question into into prompt text

context = "\n\n".join(relevant_chunks)
prompt = make_prompt(context, question_text)

In [23]:
print(prompt)

come after GAwatsonx.data CapabilitiesAI/ML at scaleBuild, train, tune, deploy, and monitor trusted AI/ML models for mission critical workloads with data in watsonx.data and ensure compliance with lineage and reproducibility of data used for AI.E.g., Presto and Spark ensure an open lakehouse architectureStreamline data engineeringReduce data pipelines, simplify data transformation, and enrich data for consumption using SQL, Python, or an AI infused conversational interface.E.g., insights

infused conversational interface.E.g., insights powered by generative AI through the power of watsonx.ai foundation models giving you the capability to span the AI lifecycle.Real-time analytics & BICombine data from existing sources with new data in watsonx.datato unlock new, faster insights without the cost and complexity of duplicating and moving data across different environments.E.g., such as Parquet, Avro, Apache ORC while leveragingApache Iceberg table format and shared metadata to share

x Scal

In [24]:
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

# Model Parameters
params = {
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MIN_NEW_TOKENS: 1,
        GenParams.MAX_NEW_TOKENS: 1000,
        GenParams.TEMPERATURE: 0.5,
}
model = Model(
        model_id='ibm/granite-13b-chat-v2', 
        params=params, credentials=creds, 
        project_id=project_id
)

# Prompt LLM
response = model.generate_text(prompt)
print(f"Question: {question_text}{response}")

Question: List some core functionalities of watsonx.data in 500 words?

Watsonx.data is a platform that offers a wide range of functionalities for managing and analyzing data, enabling organizations to build, train, and deploy trusted AI/ML models. Here are some of the core functionalities of watsonx.data:

1. Open lakehouse architecture: Watsonx.data supports an open lakehouse architecture, which allows organizations to store, manage, and analyze all their data in a single, scalable platform. This architecture combines the best of data warehousing and data lakes, providing a unified view of both structured and unstructured data.
2. SQL, Python, and AI-infused conversational interface: Watsonx.data offers a SQL interface for querying data, as well as Python and AI-infused conversational interfaces for data exploration and analysis. This flexibility enables data scientists, analysts, and other users to choose the most suitable tool for their needs, whether it's a traditional SQL query o